## Setup 

In [ ]:
import jax.numpy as jnp
import bayes3d as b
import trimesh
import os
import numpy as np
import trimesh
import jax
import cv2


# --- creating the model dir from the working directory
model_dir = os.path.join(j.utils.get_assets_dir(), "ycb_video_models/models")
print(f"{model_dir} exists: {os.path.exists(model_dir)}")
model_names = j.ycb_loader.MODEL_NAMES
model_paths = [os.path.join(model_dir,name,"textured.obj") for name in model_names]

bop_ycb_dir = os.path.join(j.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = j.ycb_loader.get_test_img('52', '1', bop_ycb_dir)
intrinsics = j.Intrinsics(
    height=rgbd.intrinsics.height,
    width=rgbd.intrinsics.width,
    fx=rgbd.intrinsics.fx, fy=rgbd.intrinsics.fx,
    cx=rgbd.intrinsics.width/2.0, cy=rgbd.intrinsics.height/2.0,
    near=0.001, far=3.0
)

In [ ]:
renderer = j.Renderer(rgbd.intrinsics, num_layers=25)
model_dir = os.path.join(j.utils.get_assets_dir(), "bop/ycbv/models")
model_names = ["obj_" + f"{str(idx+1).rjust(6, '0')}.ply" for idx in range(21)]
mesh_paths = []
for name in model_names:
    mesh_path = os.path.join(model_dir,name)
    mesh_paths.append(mesh_path)
    model_scaling_factor = 1.0/1000.0
    renderer.add_mesh_from_file(
        mesh_path,
        scaling_factor=model_scaling_factor
    )

In [ ]:
camera_pose = j.t3d.transform_from_pos_target_up(
    jnp.array([0.5, 0.5, 0.5]),
    jnp.array([0.0, 0.0, 0.0]),
    jnp.array([0.0, 0.0, 1.0]),
)
object_pose = j.t3d.inverse_pose(camera_pose)
object_pose2 = object_pose @ j.t3d.transform_from_pos(jnp.array([0.1, 0.1, 0.0]))
object_poses = jnp.array([object_pose, object_pose2])
IDX = 13
IDX2 = 14

In [ ]:
all_data = j.kubric_interface.render_multiobject_parallel([model_paths[IDX],model_paths[IDX2]], object_poses[:,None,...], intrinsics, scaling_factor=1.0, lighting=3.0) # multi img singleobj

In [ ]:
rgbd = all_data[0]
j.get_rgb_image(rgbd.rgb)

In [ ]:
# function taking the rbgd rgb and intrinstics as well as the renderer and returning the cosypose prediction 
def cosypose_pred(rgb, intrinsics, renderer):
    pred = j.cosypose_utils.cosypose_interface(rgb, j.K_from_intrinsics(intrinsics))
    pred_poses, pred_ids, pred_scores = pred['pred_poses'], pred['pred_ids'], pred['pred_scores']
    rendered = renderer.render_multiobject(jnp.array(pred_poses[0]), jnp.array(pred_ids[0]))
    return j.get_depth_image(rendered[:,:,2]) 

In [ ]:
cosypose_pred(rgbd.rgb, rgbd.intrinsics, renderer)

## Breaking CosyPose, Image Variations

1. lighting variations or illumination changes
2. Gaussian noise to rgb or low resolution (failing)
    - Reconstruction breaks down even with 0.5 scaling 
3. Partially off screen (passing)
4. Partially occluded by another object (passing)
5. Complex Backgrounds
6. Warping 
7. Others 
    6.1. Multiple Identical Objects: Passes 

### gaussian noise, low resolution 

In [ ]:
def add_gaussian_noise(img, mean=0, variance=100):
    """
    Add Gaussian noise to an image.

    Parameters:
    img (numpy.ndarray): Input image as a NumPy array.
    mean (float, optional): Mean of the Gaussian noise. Default is 0.
    variance (float, optional): Variance of the Gaussian noise. Default is 0.1.

    Returns:
    numpy.ndarray: The noisy image as a NumPy array.
    """
    # Add Gaussian noise
    noise = np.random.normal(mean, variance**0.5, img.shape)
    noisy_img = np.clip(img + noise, 0, 255).astype(np.uint8)

    return noisy_img

def make_low_resolution(img, scale_factor=0.5):
    """
    Create a low-resolution version of an image by downsampling and upsampling.

    Parameters:
    img (numpy.ndarray): Input image as a NumPy array.
    scale_factor (float, optional): The scale factor to downsample and upsample the image. Default is 0.5.

    Returns:
    numpy.ndarray: The low-resolution image as a NumPy array.
    """
    # Downsample the image
    downsampled_img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # Upsample the image
    low_res_img = cv2.resize(downsampled_img, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    return low_res_img

In [ ]:
camera_pose = j.t3d.transform_from_pos_target_up(
    jnp.array([0.5, 0.5, 0.5]),
    jnp.array([0.0, 0.0, 0.0]),
    jnp.array([0.0, 0.0, 1.0]),
)
# object poses in camera frame
object_pose = j.t3d.inverse_pose(camera_pose)

# list of positions from 0.1, 0.1 to 0.2, 0.2
poses_list = []
for i in range(3):
    for k in range(3):
        poses_list.append(object_pose @ j.t3d.transform_from_pos(jnp.array([0.1*i, 0.1*k, 0.0])))

object_poses = jnp.array(poses_list)

# testing a variety of models 
idx_list = [i for i in range(10,19)]
m_paths = [] 
for idx in idx_list:
    m_paths.append(model_paths[idx])

In [ ]:
all_data = j.kubric_interface.render_multiobject_parallel(m_paths, object_poses[:,None,...], intrinsics, scaling_factor=1.0, lighting=3.0) # multi img singleobj
rgbd = all_data[0]
j.get_rgb_image(rgbd.rgb)

In [ ]:
gauss = add_gaussian_noise(rgbd.rgb, variance=1200)
j.get_rgb_image(gauss)

In [ ]:
cosypose_pred(gauss, rgbd.intrinsics, renderer)

In [ ]:
low_res = make_low_resolution(rgbd.rgb, scale_factor=.5)
j.get_rgb_image(low_res)

In [ ]:
cosypose_pred(low_res, rgbd.intrinsics, renderer)

In [ ]:
gauss_low_res = make_low_resolution(gauss, scale_factor=.25)
low_res_gauss = add_gaussian_noise(low_res, variance=300)
j.get_rgb_image(gauss_low_res)

In [ ]:
cosypose_pred(gauss_, rgbd.intrinsics, renderer)

In [ ]:
j.get_rgb_image(low_res_gauss)

In [ ]:
pred = j.cosypose_utils.cosypose_interface(low_res_gauss, j.K_from_intrinsics(rgbd.intrinsics))
pred_poses, pred_ids, pred_scores = pred['pred_poses'], pred['pred_ids'], pred['pred_scores']
rendered = renderer.render_multiobject(jnp.array(pred_poses[0]), jnp.array(pred_ids[0]))
j.get_depth_image(rendered[:,:,2])

### Partially Occuluded

In [ ]:
import jax.numpy as jnp
import bayes3d as b
import trimesh
import os
import numpy as np
import trimesh
import jax


# --- creating the model dir from the working directory
model_dir = os.path.join(j.utils.get_assets_dir(), "ycb_video_models/models")
print(f"{model_dir} exists: {os.path.exists(model_dir)}")
model_names = j.ycb_loader.MODEL_NAMES
model_paths = [os.path.join(model_dir,name,"textured.obj") for name in model_names]

bop_ycb_dir = os.path.join(j.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = j.ycb_loader.get_test_img('52', '1', bop_ycb_dir)
intrinsics = j.Intrinsics(
    height=rgbd.intrinsics.height,
    width=rgbd.intrinsics.width,
    fx=rgbd.intrinsics.fx, fy=rgbd.intrinsics.fx,
    cx=rgbd.intrinsics.width/2.0, cy=rgbd.intrinsics.height/2.0,
    near=0.001, far=3.0
)
renderer = j.Renderer(rgbd.intrinsics, num_layers=25)
model_dir = os.path.join(j.utils.get_assets_dir(), "bop/ycbv/models")
model_names = ["obj_" + f"{str(idx+1).rjust(6, '0')}.ply" for idx in range(21)]
mesh_paths = []
for name in model_names:
    mesh_path = os.path.join(model_dir,name)
    mesh_paths.append(mesh_path)
    model_scaling_factor = 1.0/1000.0

In [ ]:
#poses
camera_pose = j.t3d.transform_from_pos_target_up(
    jnp.array([0.5, 0.5, 0.5]),
    jnp.array([0.0, 0.0, 0.0]),
    jnp.array([0.0, 0.0, 1.0]),
)
# object poses in camera frame
object_pose = j.t3d.inverse_pose(camera_pose)

# list of positions from 0.1, 0.1 to 0.2, 0.2
poses_list = []
for i in range(3):
    for k in range(3):
        poses_list.append(object_pose @ j.t3d.transform_from_pos(jnp.array([0.1*i, 0.1*k, 0.0])))

object_poses = jnp.array(poses_list)

# model paths 
# a list from one to nine 
idx_list = [i for i in range(10,19)]

#add model paths to list based on idx_list
m_paths = [] 
for idx in idx_list:
    m_paths.append(model_paths[idx])

In [ ]:
all_data = j.kubric_interface.render_multiobject_parallel(m_paths, object_poses[:,None,...], intrinsics, scaling_factor=1.0, lighting=3.0) # multi img singleobj
rgbd = all_data[0]
j.get_rgb_image(rgbd.rgb)

In [ ]:
pred = j.cosypose_utils.cosypose_interface(np.array(rgbd.rgb), j.K_from_intrinsics(rgbd.intrinsics))
pred_poses, pred_ids, pred_scores = pred['pred_poses'], pred['pred_ids'], pred['pred_scores']

In [ ]:
rendered = renderer.render_multiobject(jnp.array(pred_poses[0]), jnp.array(pred_ids[0]))
j.get_depth_image(rendered[:,:,2])